In [1]:
import nltk
import spacy
import re
import underthesea as ud # Library for Vietnamese NLP

import pandas as pd
import json

/home/tuanphan/miniconda3/envs/all_in/lib/python3.10/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/tuanphan/miniconda3/envs/all_in/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Import dataset

In [2]:
df = pd.read_json('news_dataset.json')

In [3]:
df.head(2)

,id,author,content,picture_count,processed,source,title,topic,url,crawled_at
0,218270,,"Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã c...",3,0,docbao.vn,"Tên cướp tiệm vàng tại Huế là đại uý công an, ...",Pháp luật,https://docbao.vn/phap-luat/ten-cuop-tiem-vang...,2022-08-01 09:09:22.817308
1,218269,(Nguồn: Sina),"Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số,...",1,0,vtc.vn,"Bỏ qua mạng 5G, Nga tiến thẳng từ 4G lên 6G",Sống kết nối,https://vtc.vn/bo-qua-mang-5g-nga-tien-thang-t...,2022-08-01 09:09:21.181469


In [ ]:
df.info()

# Create stopword base on dataset: Chạy colab, chạy local nếu muốn banh máy :DD

In [17]:
from collections import Counter
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor

# Hàm đếm từ cho từng phần
def count_words(texts):
    all_text = ' '.join(texts)
    normalized_text = ud.text_normalize(all_text)
    words = ud.word_tokenize(normalized_text)
    return Counter(words)

# Hàm xử lý chunk với ThreadPoolExecutor
def process_chunk(chunk):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(count_words, [text]) for text in chunk]
        results = [future.result() for future in futures]
    return sum(results, Counter())  # Gộp kết quả từ các thread

if __name__ == "__main__":
    # Giả sử df['content'] chứa 180k row
    num_cpus = 32  # Hoặc sử dụng cpu_count() để lấy số CPU trên máy

    # Chia dữ liệu thành 32 phần dựa trên số CPU
    chunk_size = (len(df) // num_cpus) + 1
    chunks = [df['content'][i:i + chunk_size] for i in range(0, len(df), chunk_size)]

    # Tạo Pool để xử lý song song từng chunk
    with Pool(processes=num_cpus) as pool:
        results = pool.map(process_chunk, chunks)

    # Gộp kết quả từ tất cả các chunk
    word_counts = Counter()
    for result in results:
        word_counts.update(result)

    # Tạo danh sách từ dừng (stopwords)
    # total_words = sum(word_counts.values())
    stopwords = {word: count for word, count in word_counts.items()}


    # Lưu bộ từ dừng dưới dạng JSON
    with open('stopwords.json', 'w', encoding='utf-8') as f:
        json.dump(stopwords, f, ensure_ascii=False, indent=4)


# 2. Xử lý với 1 content mẫu

In [5]:
text = df['content'][0]
text

"Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã có thông tin ban đầu về vụ nổ súng,cướp tiệm vàng tại chợ Đông Ba nằm trên đường Trần Hưng Đạo (TP Huế, tỉnh Thừa Thiên - Huế). Thông Sài Gòn Giải Phóng, khoảng 12h30' ngày 31/7, một đối tượng sử dụng súng AK bất ngờ xông vào tiệm vàng Hoàng Đức và Thái Lợi (phía trước chợ Đông Ba) rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. Sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu Gia Hội, cách khu vực gây án khoảng 300m. Giám đốc Công an tỉnh Thừa Thiên – Huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm Công an tỉnh và Công an TP Huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. Hàng trăm tiểu thương trong chợ Đông Ba và người dân gần cầu Gia Hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. Tuy nhiên, thấy vàng bị ném ra đường, nhiều người đua nhau nhặt, tạo cảnh nhốn

- Remove phones, emails, URL, special characters
(using Re)

- Tokenization

In [ ]:
# from underthesea import sent_tokenize,word_tokenize
ud.sent_tokenize(text)

In [ ]:
def preprocess(text):
    
    # Text normalization
    text_normalized = ud.text_normalize(text)

    # Note: Nếu là tiếng Anh thì đoạn này sẽ làm Lemma hoặc Stemm, 
    # Còn với tiếng Việt thì không cần

    # # Sentence segmentation
    # sents = ud.sent_tokenize(text_normalized)

    # Word tokenization
    tokens = ud.word_tokenize(text_normalized)

    # Remove stop words,number,noise...
    def removed_tokens(word_tokens):
        return [token for token in word_tokens if ]

    # Combine again to sentence
    
    # POS tagging
    pos_list = ud.pos_tag(text_normalized)

    



    

    

In [43]:
text

"Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã có thông tin ban đầu về vụ nổ súng,cướp tiệm vàng tại chợ Đông Ba nằm trên đường Trần Hưng Đạo (TP Huế, tỉnh Thừa Thiên - Huế). Thông Sài Gòn Giải Phóng, khoảng 12h30' ngày 31/7, một đối tượng sử dụng súng AK bất ngờ xông vào tiệm vàng Hoàng Đức và Thái Lợi (phía trước chợ Đông Ba) rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. Sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu Gia Hội, cách khu vực gây án khoảng 300m. Giám đốc Công an tỉnh Thừa Thiên – Huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm Công an tỉnh và Công an TP Huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. Hàng trăm tiểu thương trong chợ Đông Ba và người dân gần cầu Gia Hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. Tuy nhiên, thấy vàng bị ném ra đường, nhiều người đua nhau nhặt, tạo cảnh nhốn

In [4]:
import underthesea as ud

In [5]:
ud.word_tokenize(ud.text_normalize(''.join(df['content'][0])))

NameError: name 'df' is not defined